<a href="https://colab.research.google.com/github/Rinkal-27/LGMVIP--DataScience/blob/main/Stock_Market_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
url = "https://raw.githubusercontent.com/mwitiderrick/stockprice/master/NSE-TATAGLOBAL.csv"

dataset = pd.read_csv(url)

print(dataset.head())

         Date    Open    High     Low    Last   Close  Total Trade Quantity  \
0  2018-09-28  234.05  235.95  230.20  233.50  233.75               3069914   
1  2018-09-27  234.55  236.80  231.10  233.80  233.25               5082859   
2  2018-09-26  240.00  240.00  232.50  235.00  234.25               2240909   
3  2018-09-25  233.30  236.75  232.00  236.25  236.10               2349368   
4  2018-09-24  233.55  239.20  230.75  234.00  233.30               3423509   

   Turnover (Lacs)  
0          7162.35  
1         11859.95  
2          5248.60  
3          5503.90  
4          7999.55  


In [ ]:
data = dataset[['Date', 'Close']]

data = data.sort_values('Date')

data.set_index('Date', inplace=True)

In [ ]:
# Normalizing the data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

In [ ]:
# Split the data into training and testing sets
train_size = int(len(data) * 0.8)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

In [ ]:
# Function to create sequences and labels from the data
def create_sequences(data, sequence_length):
    X, y = [], []
    for i in range(len(data)-sequence_length):
        X.append(data[i:i+sequence_length])
        y.append(data[i+sequence_length])
    return np.array(X), np.array(y)

In [ ]:
# Set the sequence length
sequence_length = 100
X_train, y_train = create_sequences(train_data, sequence_length)

In [ ]:
# Create sequences and labels for the testing set
X_test, y_test = create_sequences(test_data, sequence_length)

# Reshape the input data for LSTM
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

In [ ]:
# Build the Stacked LSTM model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
24/24 [==============================] - 19s 174ms/step - loss: 0.0080
Epoch 2/10
24/24 [==============================] - 5s 227ms/step - loss: 0.0013
Epoch 3/10
24/24 [==============================] - 4s 173ms/step - loss: 9.6242e-04
Epoch 4/10
24/24 [==============================] - 4s 172ms/step - loss: 9.0078e-04
Epoch 5/10
24/24 [==============================] - 5s 230ms/step - loss: 8.3751e-04
Epoch 6/10
24/24 [==============================] - 4s 176ms/step - loss: 8.0799e-04
Epoch 7/10
24/24 [==============================] - 4s 173ms/step - loss: 7.7703e-04
Epoch 8/10
24/24 [==============================] - 5s 227ms/step - loss: 7.3398e-04
Epoch 9/10
24/24 [==============================] - 4s 170ms/step - loss: 7.2060e-04
Epoch 10/10
24/24 [==============================] - 5s 194ms/step - loss: 6.7399e-04


In [ ]:
# Making predictions on the testing set
predictions = model.predict(X_test)

10/10 [==============================] - 2s 41ms/step


In [ ]:
# Inversing transform the predictions and actual values
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)


In [ ]:
# Calculate the root mean squared error (RMSE)
rmse = np.sqrt(np.mean((predictions - y_test)**2))
print('RMSE:', rmse)

RMSE: 22.24501481616915
